<a href="https://colab.research.google.com/github/priyanshupriyank04/Trading_system_5WMA_reversal/blob/main/5wma_reversal_nifty50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install missing libraries
!pip install --quiet kiteconnect pandas matplotlib numpy

# For data handling and manipulation
import pandas as pd
import numpy as np

# For working with dates and times
from datetime import datetime, timedelta

# For connecting to Kite API
from kiteconnect import KiteConnect

# For managing and storing data in Google Drive
from google.colab import drive

# For data visualization (optional, useful during backtesting analysis)
import matplotlib.pyplot as plt

# Mount Google Drive for persistent storage
drive.mount('/content/drive', force_remount=True)

print("All libraries installed and imported successfully!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.5/771.5 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.5/254.5 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 5.8 MB/s eta 0:00:00
Mounted at /content/drive
All libraries installed and imported successfully!


Access and request token

In [20]:
API_KEY = "8re7mjcm2btaozwf"
API_SECRET = "fw8gm7wfeclcic9rlkp0tbzx4h2ss2n1"
REDIRECT_URL = "https://example.com"


kite = KiteConnect(api_key=API_KEY)


login_url = kite.login_url()
print(f"Login to your Kite account using this URL: {login_url}")


request_token = input("Paste the request token from the URL here: ")


try:
    data = kite.generate_session(request_token=request_token, api_secret=API_SECRET)
    access_token = data["access_token"]
    print(f"Access Token: {access_token}")


    kite.set_access_token(access_token)
    print("Kite Connect is now initialized and ready to use!")
except Exception as e:
    print(f"Error generating access token: {e}")

Login to your Kite account using this URL: https://kite.zerodha.com/connect/login?api_key=8re7mjcm2btaozwf&v=3
Paste the request token from the URL here: mPaczi8qkegIw7A5iCY95csPoc9Cm0OD
Access Token: m1kDlJt22DbmP72boONZ6HipJpVWj8Ai
Kite Connect is now initialized and ready to use!


In [21]:
def get_instrument_token(kite, tradingsymbol="NIFTY", exchange="NSE"):
    try:
        # Fetch all instruments for the given exchange
        instruments = kite.instruments(exchange)

        # Loop through the instruments and find the matching trading symbol
        for instrument in instruments:
            if instrument['tradingsymbol'] == tradingsymbol:
                print(f"Instrument Token for {tradingsymbol}: {instrument['instrument_token']}")
                return instrument['instrument_token']

        print(f"Instrument Token for {tradingsymbol} not found.")
        return None
    except Exception as e:
        print(f"Error fetching instrument token: {e}")
        return None

# Example Usage
nifty50_token = get_instrument_token(kite, tradingsymbol="NIFTY 50", exchange="NSE")

Instrument Token for NIFTY 50: 256265


Fetch data & save to csv file in drive

In [22]:
import os

def fetch_monthly_data(kite, instrument_token, year, output_csv_path):

    all_data = []

    # Loop through each month
    for month in range(1, 13):
        start_date = datetime(year, month, 1)

        # Calculate the end date of the month
        next_month = start_date + timedelta(days=32)
        end_date = datetime(next_month.year, next_month.month, 1) - timedelta(days=1)

        # Ensure end_date does not exceed today's date
        today = datetime.now()
        if end_date > today:
            end_date = today - timedelta(days=1)

        print(f"Fetching data from {start_date.date()} to {end_date.date()}...")

        try:
            # Fetch historical data for the current month
            monthly_data = kite.historical_data(
                instrument_token=instrument_token,
                from_date=start_date.strftime("%Y-%m-%d"),
                to_date=end_date.strftime("%Y-%m-%d"),
                interval="minute"
            )

            # Convert to DataFrame and add to the list
            monthly_df = pd.DataFrame(monthly_data)
            print("Columns in monthly_df:", monthly_df.columns)
            monthly_df['date'] = pd.to_datetime(monthly_df['date'])
            monthly_df.set_index('date', inplace=True)

            # Use lowercase column names for OHLC
            monthly_df = monthly_df[['open', 'high', 'low', 'close']]
            all_data.append(monthly_df)

            print(f"Fetched {len(monthly_df)} rows for {start_date.strftime('%B %Y')}.")
        except Exception as e:
            print(f"Error fetching data for {start_date.strftime('%B %Y')}: {e}")

        # Break the loop if we reach today's date
        if end_date >= today:
            break

    # Combine all monthly data
    if all_data:
        combined_data = pd.concat(all_data)

        # Save the combined data to CSV
        combined_data.to_csv(output_csv_path)
        print(f"Data saved to {output_csv_path}.")
    else:
        print("No data fetched.")

# Path to save the CSV in Google Drive
output_csv_path = "/content/drive/My Drive/nifty50_1min_data_2024.csv"

# Fetch the data for 2024 and save to Google Drive
fetch_monthly_data(kite, nifty50_token, year=2024, output_csv_path=output_csv_path)


Fetching data from 2024-01-01 to 2024-01-31...
Columns in monthly_df: Index(['date', 'open', 'high', 'low', 'close', 'volume'], dtype='object')
Fetched 8250 rows for January 2024.
Fetching data from 2024-02-01 to 2024-02-29...
Columns in monthly_df: Index(['date', 'open', 'high', 'low', 'close', 'volume'], dtype='object')
Fetched 7875 rows for February 2024.
Fetching data from 2024-03-01 to 2024-03-31...
Columns in monthly_df: Index(['date', 'open', 'high', 'low', 'close', 'volume'], dtype='object')
Fetched 6855 rows for March 2024.
Fetching data from 2024-04-01 to 2024-04-30...
Columns in monthly_df: Index(['date', 'open', 'high', 'low', 'close', 'volume'], dtype='object')
Fetched 7494 rows for April 2024.
Fetching data from 2024-05-01 to 2024-05-31...
Columns in monthly_df: Index(['date', 'open', 'high', 'low', 'close', 'volume'], dtype='object')
Fetched 7980 rows for May 2024.
Fetching data from 2024-06-01 to 2024-06-30...
Columns in monthly_df: Index(['date', 'open', 'high', 'low',

Prev close & Prev open

In [87]:
def add_prev_close_column(file_path):

    # Load the existing CSV
    df = pd.read_csv(file_path)

    # Add 'prev_close' column
    df['prev_close'] = df['high'].shift(1)
    df.loc[0,'prev_close'] = df.loc[0,'high']

    # Save back to the same file
    df.to_csv(file_path, index=False)
    print(f"'prev_close' column added successfully and saved to {file_path}.")

def add_prev_open_column(file_path):

    # Load the existing CSV
    df = pd.read_csv(file_path)

    # Add 'prev_open' column
    df['prev_open'] = df['open'].shift(1)
    df.loc[0,'prev_open'] = df.loc[0,'open']

    # Save back to the same file
    df.to_csv(file_path, index=False)
    print(f"'prev_open' column added successfully and saved to {file_path}.")

#Path to the Existing CSV in Google Drive
file_path = "/content/drive/My Drive/nifty50_1min_data_2024.csv"

#Add 'prev_close' and 'prev_open' Columns
add_prev_close_column(file_path)
add_prev_open_column(file_path)


'prev_close' column added successfully and saved to /content/drive/My Drive/nifty50_1min_data_2024.csv.
'prev_open' column added successfully and saved to /content/drive/My Drive/nifty50_1min_data_2024.csv.


below prev count

In [89]:
def add_below_prev_count_column(file_path):

    df = pd.read_csv(file_path)


    if 'prev_close' not in df.columns:
        raise ValueError("The 'prev_close' column is missing. Please add it before running this function.")


    below_prev_count = []
    count = 0

    for i in range(len(df)):
        if i == 0:

            below_prev_count.append(0)
        else:
            if df['high'].iloc[i] < df['prev_close'].iloc[i]:
                count += 1
            else:
                count = 0
            below_prev_count.append(count)


    df['below_prev_count'] = below_prev_count


    df.to_csv(file_path, index=False)
    print(f"'below_prev_count' column added successfully and saved to {file_path}.")
add_below_prev_count_column(file_path)


'below_prev_count' column added successfully and saved to /content/drive/My Drive/nifty50_1min_data_2024.csv.


5WMA

In [90]:
def add_5wma_column(file_path):


    df = pd.read_csv(file_path)


    if 'close' not in df.columns:
        raise ValueError("The 'close' column is missing. Please add it before running this function.")

    # Calculate 5WMA
    def calculate_5wma(series):
        weights = np.arange(1, 6)
        return np.dot(series, weights) / weights.sum()

    df['5wma'] = df['close'].rolling(window=5).apply(calculate_5wma, raw=True)

    # Fill the first 4 rows with 5EMA
    df['5ema'] = df['close'].ewm(span=5, adjust=False).mean()
    df.loc[:4, '5wma'] = df.loc[:4, '5ema']


    df.drop(columns=['5ema'], inplace=True)


    df.to_csv(file_path, index=False)
    print(f"'5wma' column added successfully and saved to {file_path}.")

add_5wma_column(file_path)

'5wma' column added successfully and saved to /content/drive/My Drive/nifty50_1min_data_2024.csv.


Fill nan cols

In [91]:
def fill_nan_with_previous(file_path):

    df = pd.read_csv(file_path)


    df.ffill(inplace=True)


    df.to_csv(file_path, index=False)
    print(f"NaN values filled with the value above and saved to {file_path}.")

fill_nan_with_previous(file_path)

NaN values filled with the value above and saved to /content/drive/My Drive/nifty50_1min_data_2024.csv.


Backtest Strategy and results

In [7]:
def backtest_strategy(file_path):

    df = pd.read_csv(file_path)


    required_columns = ['close', 'open', 'low', 'high', 'below_prev_count', '5wma']
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"The required column '{col}' is missing from the dataset.")


    position_open = False
    total_profit = 0
    total_win_trades = 0
    total_loss_trades = 0
    trade_log = []  # To keep track of individual trade results


    for i in range(1, len(df) - 1):
        if not position_open:
            # Check for alert candle
            if (
                df['below_prev_count'].iloc[i] == 2 and
                df['high'].iloc[i] > df['5wma'].iloc[i]

            ):
                # Trade parameters
                trigger_price = df['open'].iloc[i + 1]  # Next candle open price
                target = trigger_price + 8 # Target price
                stop_loss = df['low'].iloc[i]  # Stop-loss is low of alert candle
                position_open = True
                trade_log.append({
                    "entry_price": trigger_price,
                    "stop_loss": stop_loss,
                    "target": target,
                    "result": None,
                })

        elif position_open:
            # Check trigger candle for exit conditions
            trigger_high = df['high'].iloc[i]
            trigger_low = df['low'].iloc[i]

            if trigger_high >= trade_log[-1]['target']:
                # Target hit
                profit = trade_log[-1]['target'] - trade_log[-1]['entry_price']
                total_profit += profit
                total_win_trades += 1
                trade_log[-1]['result'] = 'win'
                trade_log[-1]['exit_price'] = trade_log[-1]['target']
                position_open = False

            elif trigger_low <= trade_log[-1]['stop_loss']:
                # Stop-loss hit
                loss = trade_log[-1]['entry_price'] - trade_log[-1]['stop_loss']
                total_profit -= loss
                total_loss_trades += 1
                trade_log[-1]['result'] = 'loss'
                trade_log[-1]['exit_price'] = trade_log[-1]['stop_loss']
                position_open = False

    # Calculate metrics
    total_trades = total_win_trades + total_loss_trades
    accuracy = (total_win_trades / total_trades) * 100 if total_trades > 0 else 0

    # Return results
    return {
        "total_profit": total_profit,
        "total_win_trades": total_win_trades,
        "total_loss_trades": total_loss_trades,
        "accuracy": accuracy,
        "trade_log": trade_log,  # Optional: detailed trade log
    }


file_path = "/content/drive/My Drive/nifty50_1min_data_2024.csv"
results = backtest_strategy(file_path)

# Print results
print(f"Total Profit: {results['total_profit']}")
print(f"Total Win Trades: {results['total_win_trades']}")
print(f"Total Loss Trades: {results['total_loss_trades']}")
print(f"Accuracy: {results['accuracy']:.2f}%")


Total Profit: 6575.300000000196
Total Win Trades: 2940
Total Loss Trades: 6111
Accuracy: 32.48%
